# Learning Box Embeddings with Example

This tutorial outlines the different functionalities available within the Box Embeddings package

### 0. Installing the package on your machine

*If you have the repo cloned*
```
pip install --editable . --user
```

### A. Initialize a box tensor and check its parameters

#### Standard Box Tensor
To represent a Tensor as a Box, we use the class `BoxTensor`. The necessary parameter is `data` (a tensor)

In [2]:
from box_embeddings.parameterizations.box_tensor import *
# Let's create a toy example
x_min = [-2.0]*50
x_max = [0.0]*50
data_x = torch.tensor([[[1,2],[-1,5]], [[0,2],[-2,3]], [[-3,3],[-2,4]]])
#tensor = torch.tensor(np.random.rand(3, 2, 2))
box_1 = BoxTensor(data_x)
box_1

BoxTensor(tensor([[[ 1,  2],
         [-1,  5]],

        [[ 0,  2],
         [-2,  3]],

        [[-3,  3],
         [-2,  4]]]))

We can use several methods to look at the parameters of our box, such as

In [3]:
# Lower left coordinate
print(box_1.z)
# Top right coordinate
print(box_1.Z)
# Center coordinate
print(box_1.centre)

tensor([[ 1,  2],
        [ 0,  2],
        [-3,  3]])
tensor([[-1,  5],
        [-2,  3],
        [-2,  4]])
tensor([[ 0.0000,  3.5000],
        [-1.0000,  2.5000],
        [-2.5000,  3.5000]])


Let's broadcast our box to a new shape.. Broadcasting is often needed for different arithmetic operations. The function we
will use is `broadcast()`, and the required parameter is `target_shape=()`, which specify the new shape
for the box. This is very similar to `numpy.broadcast_to()`

In [4]:
data = torch.tensor([[[1, 2, 3], [3, 4, 6]],
          [[5, 6, 8], [7, 9, 5]]])
box = BoxTensor(data)
print('previous shape is:', box.box_shape)
box.broadcast(target_shape=(2, 1, 3))
print('after broadcasting:', box.box_shape)

previous shape is: (2, 3)
after broadcasting: (2, 1, 3)


### 2. Box Volume
To calculate the volume of a box, we can use either the `soft_volume`, or the Bessel volume via `bessel_volume_approx`.
To ensure numerical stability, we can use the log version via `log_soft_volume` or `log_bessel_volume_approx`

In [34]:
from box_embeddings.modules.volume.soft_volume import soft_volume

# Create data as tensors, and initialize a box
data = torch.tensor([[-2.0]*100, [0.0]*100])
box_1 = BoxTensor(data)

# Logged Soft volume
print(soft_volume(box_1))

#Soft volume
print(soft_volume(box_1, log_scale=False))

tensor(75.4679)
tensor(5.9605e+32)


### 3. Box Intersection

To calculate the intersection of two boxes (which yields a box), we can use either `hard_intersection` or
`gumbel_intersection`

In [35]:
from box_embeddings.modules.intersection import hard_intersection, gumbel_intersection

# Create data as tensors, and initialize two boxes, box_1 and box_2
x_min = [-2.0]*100
x_max = [0.0]*100
data_x = torch.tensor([x_min, x_max])
box_1 = BoxTensor(data_x)

y_min = [1/n for n in range(1, 101)]
y_max = [1 - k for k in reversed(y_min)]
data_y = torch.tensor([y_min, y_max], requires_grad=True)
box_2 = BoxTensor(data_y)

# Intersection of box_1 and box_2
print(hard_intersection(box_1, box_2))

# Gumbel intersection of box_1 and box_2
print(gumbel_intersection(box_1, box_2))

BoxTensor(z=tensor([1.0000, 0.5000, 0.3333, 0.2500, 0.2000, 0.1667, 0.1429, 0.1250, 0.1111,
        0.1000, 0.0909, 0.0833, 0.0769, 0.0714, 0.0667, 0.0625, 0.0588, 0.0556,
        0.0526, 0.0500, 0.0476, 0.0455, 0.0435, 0.0417, 0.0400, 0.0385, 0.0370,
        0.0357, 0.0345, 0.0333, 0.0323, 0.0312, 0.0303, 0.0294, 0.0286, 0.0278,
        0.0270, 0.0263, 0.0256, 0.0250, 0.0244, 0.0238, 0.0233, 0.0227, 0.0222,
        0.0217, 0.0213, 0.0208, 0.0204, 0.0200, 0.0196, 0.0192, 0.0189, 0.0185,
        0.0182, 0.0179, 0.0175, 0.0172, 0.0169, 0.0167, 0.0164, 0.0161, 0.0159,
        0.0156, 0.0154, 0.0152, 0.0149, 0.0147, 0.0145, 0.0143, 0.0141, 0.0139,
        0.0137, 0.0135, 0.0133, 0.0132, 0.0130, 0.0128, 0.0127, 0.0125, 0.0123,
        0.0122, 0.0120, 0.0119, 0.0118, 0.0116, 0.0115, 0.0114, 0.0112, 0.0111,
        0.0110, 0.0109, 0.0108, 0.0106, 0.0105, 0.0104, 0.0103, 0.0102, 0.0101,
        0.0100], grad_fn=<MaximumBackward>),
Z=tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0

### 4. Box Training
In the following example, we train a simple box `box_2` to require it to be completely contained inside another box
`box_1`. The training loop returns the best `box_2`.

In [37]:
from box_embeddings.modules.volume.volume import Volume
from box_embeddings.modules.intersection import Intersection

# Initialize two boxes
x_min = [-2.0 for n in range(1, 101)]
x_max = [0.0 for k in reversed(x_min)]
data_x = torch.tensor([x_min, x_max], requires_grad=True)
box_1 = BoxTensor(data_x)

y_min = [1/n for n in range(1, 101)]
y_max = [1 - k for k in reversed(y_min)]
data_y = torch.tensor([y_min, y_max], requires_grad=True)
box_2 = BoxTensor(data_y)

# Training loop
learning_rate = 0.1

def train(box_1, box_2, optimizer, epochs=1):
    best_loss = int()
    best_box_2 = None
    box_vol = Volume()
    box_int = Intersection()
    for e in range(epochs):
        loss = box_vol.forward(box_2) - box_vol.forward(box_int._forward(box_1, box_2))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if best_loss < loss.item():
            best_loss = loss.item()
            best_box_2 = box_2
        print('Iteration %d, loss = %.4f' % (e, loss.item()))
    return best_box_2

optimizer =  torch.optim.SGD([data_y], lr=learning_rate)
train(box_1, box_2, optimizer, epochs=20)

Iteration 0, loss = 2924.1987
Iteration 1, loss = 2894.2686
Iteration 2, loss = 2730.0720
Iteration 3, loss = 1665.2723
Iteration 4, loss = 0.0000
Iteration 5, loss = 0.0000
Iteration 6, loss = 0.0000
Iteration 7, loss = 0.0000
Iteration 8, loss = 0.0000
Iteration 9, loss = 0.0000
Iteration 10, loss = 0.0000
Iteration 11, loss = 0.0000
Iteration 12, loss = 0.0000
Iteration 13, loss = 0.0000
Iteration 14, loss = 0.0000
Iteration 15, loss = 0.0000
Iteration 16, loss = 0.0000
Iteration 17, loss = 0.0000
Iteration 18, loss = 0.0000
Iteration 19, loss = 0.0000


BoxTensor(tensor([[ 1.0000e+00,  1.9288e+00,  7.6992e-01,  2.9159e+00,  1.1253e+00,
          8.8667e-01,  7.7884e-01,  7.1439e-01,  6.7064e-01,  6.3867e-01,
          6.1416e-01,  5.9472e-01,  5.7889e-01,  5.6574e-01,  5.5464e-01,
          5.4514e-01,  5.3692e-01,  5.2974e-01,  5.2341e-01,  4.8874e+01,
          1.0975e+01,  6.6676e+00,  5.0054e+00,  4.1244e+00,  3.5791e+00,
          3.2086e+00,  2.9408e+00,  2.7385e+00,  2.5807e+00,  2.4544e+00,
          2.3513e+00,  2.2658e+00,  2.1939e+00,  2.1330e+00,  2.0809e+00,
          2.0361e+00,  1.9973e+00,  1.9637e+00,  1.9345e+00,  1.9091e+00,
          1.8871e+00,  1.8681e+00,  1.8518e+00,  1.8380e+00,  1.8263e+00,
          1.8168e+00,  1.8093e+00,  1.8036e+00,  1.7997e+00,  1.7976e+00,
          1.7972e+00,  1.7985e+00,  1.8016e+00,  1.8065e+00,  1.8133e+00,
          1.8220e+00,  1.8328e+00,  1.8458e+00,  1.8613e+00,  1.8794e+00,
          1.9005e+00,  1.9250e+00,  1.9533e+00,  1.9859e+00,  2.0237e+00,
          2.0675e+00,  2.118